# APIM ❤️ OpenAI

## Token Metrics Emitting lab
![flow](../../images/token-metrics-emitting.gif)

Playground to try the [emit token metric policy](https://learn.microsoft.com/azure/api-management/azure-openai-emit-token-metric-policy). The policy sends metrics to Application Insights about consumption of large language model tokens through Azure OpenAI Service APIs.

Notes:
- Token count metrics include: Total Tokens, Prompt Tokens, and Completion Tokens.
- This policy supports OpenAI response streaming! Use the [streaming tool](../../tools/streaming.ipynb) to test and troubleshoot response streaming.
- Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior and troubleshoot the [policy](policy.xml).

[View policy configuration](policy.xml)

### Result
![result](result.png)

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Verify the Azure CLI and the connected Azure subscription](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [🧪 Test the API using a direct HTTP call](#requests)
- [🧪 Execute multiple runs for each subscription using the Azure OpenAI Python SDK](#sdk)
- [🔍 Analyze Application Insights custom metrics with a KQL query](#kql)
- [🔍 Plot the custom metrics results](#plot)
- [🔍 See the metrics on the Azure Portal](#portal)
- [🗑️ Clean up resources](#clean)

### Prerequisites
- [Python 3.9 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access)
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

apim_sku = 'Basicv2'

openai_resources = [
    {"name": "openai1", "location": "swedencentral"}
]

openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"

utils.print_ok('Notebook initiaized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")
if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    subscription_id = output.json_data['id']
    tenant_id = output.json_data['tenantId']

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# create the resource group if doesn't exist
utils.create_resource_group(True, resource_group_name, resource_group_location)

# update the APIM policy file before the deployment
policy_xml = None
with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    if "{backend-id}" in policy_template_xml:
        policy_xml = policy_template_xml.replace("{backend-id}", str("openai-backend-pool" if len(openai_resources) > 1 else openai_resources[0].get("name")))                
    policy_xml_file.close()
if policy_xml is not None:
    open("policy.xml", 'w').write(policy_xml)

# define the BICEP parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIAPIVersion": { "value": openai_api_version }
    }
}

# write the parameters to a file 
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json", 
                f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")
open("policy.xml", 'w').write(policy_template_xml)


<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")
if output.success and output.json_data:
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_subscription1_key = utils.get_deployment_output(output, 'apimSubscription1Key', 'APIM Subscription 1 Key (masked)', True)
    apim_subscription2_key = utils.get_deployment_output(output, 'apimSubscription2Key', 'APIM Subscription 2 Key (masked)', True)
    apim_subscription3_key = utils.get_deployment_output(output, 'apimSubscription3Key', 'APIM Subscription 3 Key (masked)', True)
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')




<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior and troubleshoot the [policy](policy.xml).

In [ ]:
# %load ../../shared/snippets/api-http-requests.py
import json
import requests
import time

runs = 10
sleep_time_ms = 100
url = f"{apim_resource_gateway_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}"
api_runs = []
messages = {"messages": [
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}

# Initialize a session for connection pooling
session = requests.Session()
# Set default headers
session.headers.update({
    'api-key': apim_subscription1_key,
    'x-user-id': 'alex'
})

try:
    for i in range(runs):
        print(f"▶️ Run {i+1}/{runs}:")

        start_time = time.time()
        response = session.post(url, json = messages)
        response_time = time.time() - start_time
        print(f"⌚ {response_time:.2f} seconds")

        # Check the response status code and apply formatting
        if 200 <= response.status_code < 300:
            status_code_str = f"\x1b[1;32m{response.status_code} - {response.reason}\x1b[0m" # Bold and green
        elif response.status_code >= 400:
            status_code_str = f"\x1b[1;31m{response.status_code} - {response.reason}\x1b[0m" # Bold and red
        else:
            status_code_str = str(response.status_code)  # No formatting

        # Print the response status with the appropriate formatting
        print(f"Response status: {status_code_str}")
        print(f"Response headers: {json.dumps(dict(response.headers), indent = 4)}")

        if "x-ms-region" in response.headers:
            print(f"x-ms-region: \x1b[1;32m{response.headers.get("x-ms-region")}\x1b[0m") # this header is useful to determine the region of the backend that served the request
            api_runs.append((response_time, response.headers.get("x-ms-region")))

        if (response.status_code == 200):
            data = json.loads(response.text)
            print(f"Token usage: {json.dumps(dict(data.get("usage")), indent = 4)}\n")
            print(f"💬 {data.get("choices")[0].get("message").get("content")}\n")
        else:
            print(response.text)

        time.sleep(sleep_time_ms/1000)
finally:
    # Close the session to release the connection
    session.close()


<a id='sdk'></a>
### 🧪 Execute multiple runs for each subscription using the Azure OpenAI Python SDK

We will send requests for each subscription. Adjust the `sleep_time_ms` and the number of `runs` to your test scenario.


In [ ]:
import time
from openai import AzureOpenAI

runs = 10
sleep_time_ms = 100

clients = [
    AzureOpenAI(
        azure_endpoint = apim_resource_gateway_url,
        api_key = apim_subscription1_key,
        api_version = openai_api_version
    ),
    AzureOpenAI(
        azure_endpoint = apim_resource_gateway_url,
        api_key = apim_subscription2_key,
        api_version = openai_api_version
    ),
    AzureOpenAI(
        azure_endpoint = apim_resource_gateway_url,
        api_key = apim_subscription3_key,
        api_version = openai_api_version
    )
]

messages = [
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]

for i in range(runs):
    print(f"▶️ Run {i+1}/{runs}:")

    for j in range(0, 3):
        response = clients[j].chat.completions.create(model = openai_model_name, messages = messages, extra_headers={"x-user-id": "alex"}) # type: ignore
        print(f"💬 Subscription {j+1}: {response.choices[0].message.content}")

    print()

    time.sleep(sleep_time_ms/1000)


<a id='kql'></a>
### 🔍 Analyze Application Insights custom metrics with a KQL query

With this query you can get the custom metrics that were emitted by Azure APIM. Note that it may take a few minutes for data to become available.

In [ ]:
# type: ignore

import pandas as pd

query = "\"" + "customMetrics \
| where name == 'Total Tokens' \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend clientIP = tostring(parsedCustomDimensions.['Client IP']) \
| extend apiId = tostring(parsedCustomDimensions.['API ID']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription ID']) \
| extend UserId = tostring(parsedCustomDimensions.['User ID']) \
| project timestamp, value, clientIP, apiId, apimSubscription, UserId \
| order by timestamp asc" + "\""

# If the query fails, replace {resource_group_name} with the name of the resource group where the Application Insights resource is deployed.
result_stdout = ! az monitor app-insights query --app {app_insights_name} -g {resource_group_name} --analytics-query {query}
print(result_stdout)
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
df = pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%H:%M')
df


<a id='plot'></a>
### 🔍 Plot the custom metrics results

In [ ]:
# plot the results
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = [15, 7]
ax = df.plot(kind = 'line', x = 'timestamp', y = 'value', legend = False)
plt.title('Total token usage over time')
plt.xlabel('Time')
plt.ylabel('Tokens')

plt.show()

<a id='portal'></a>
### 🔍 See the metrics on the Azure Portal

Open the Application Insights resource, navigate to the Metrics blade, then select the defined namespace (openai). Choose the metric "Total Tokens" with a Sum aggregation. Then, apply splitting by 'Subscription Id' to view values for each dimension.

![result](result.png)


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.